# Advent of Code 2024 - Day 6

In [18]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from pydantic import BaseModel, field_validator

In [2]:
# Load input data
input_file = os.path.join("/home/jits/adventOfCode/2024/day6", "day6.txt")
with open(input_file, "r") as file:
    data = file.read().strip()
test_input_file = os.path.join("/home/jits/adventOfCode/2024/day6", "day6test.txt")
with open(test_input_file, "r") as test_file:
    test_data = test_file.read().strip()

# Example usage
print(test_data)  # Print the test data

....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...


## Process Input

In [3]:
# Process the input data
def process_input(data):
    return data.split("\n")
processed_data = process_input(test_data)
print(processed_data)


['....#.....', '.........#', '..........', '..#.......', '.......#..', '..........', '.#..^.....', '........#.', '#.........', '......#...']


## Part 1

In [4]:
class Position(BaseModel):
    x: int
    y: int
    direction: str

    @field_validator('direction')
    def validate_direction(cls, v):
        allowed_directions = {'up', 'right', 'down', 'left'}
        if v not in allowed_directions:
            raise ValueError(f"Invalid direction: {v}")
        return v
    
    def turn_right(self):
        if self.direction == 'up':
            return Position(x=self.x, y=self.y, direction='right')
        if self.direction == 'right':
            return Position(x=self.x, y=self.y, direction='down')
        if self.direction == 'down':
            return Position(x=self.x, y=self.y, direction='left')
        if self.direction == 'left':
            return Position(x=self.x, y=self.y, direction='up')

    def move(self, maze):
        """
        Move the position in the maze
        :param maze: list of strings
        :return: new position, has_finished
        """
        if self.direction == 'up':
            # check if can move up
            if self.y == 0:
                return None, True
            elif maze[self.y - 1][self.x] == '#':
                # can't move up, turn right
                return self.turn_right(), False
            else:
                return Position(x=self.x, y=self.y - 1, direction='up'), False
        if self.direction == 'right':
            # check if can move right
            if self.x == len(maze[self.y]) - 1:
                return None, True
            elif maze[self.y][self.x + 1] == '#':
                # can't move right, turn right
                return self.turn_right(), False
            else:
                return Position(x=self.x + 1, y=self.y, direction='right'), False
        if self.direction == 'down':
            # check if can move down
            if self.y == len(maze) - 1:
                return None, True
            elif maze[self.y + 1][self.x] == '#':
                # can't move down, turn right
                return self.turn_right(), False
            else:
                return Position(x=self.x, y=self.y + 1, direction='down'), False
        if self.direction == 'left':
            # check if can move left
            if self.x == 0:
                return None, True
            elif maze[self.y][self.x - 1] == '#':
                # can't move left, turn right
                return self.turn_right(), False
            else:
                return Position(x=self.x - 1, y=self.y, direction='left'), False

    def __hash__(self):
        return hash((self.x, self.y, self.direction))

    def __eq__(self, other):
        return (self.x, self.y, self.direction) == (other.x, other.y, other.direction)

    def __repr__(self):
        return f"({self.x}, {self.y}, {self.direction})"
    
def count_distinct_positions(positions):
    positions_without_direction = {(p.x, p.y) for p in positions}
    return len(positions_without_direction)


def get_start_pos(maze):
    for y, row in enumerate(maze):
        if '^' in row:
            x = row.index('^')
            return Position(x=x, y=y, direction='up')
        elif 'v' in row:
            x = row.index('v')
            return Position(x=x, y=y, direction='down')
        elif '<' in row:
            x = row.index('<')
            return Position(x=x, y=y, direction='left')
        elif '>' in row:
            x = row.index('>')
            return Position(x=x, y=y, direction='right')


In [ ]:
maze = process_input(data)

start_pos = get_start_pos(maze)

visited = [start_pos]

current_pos = start_pos
while True:
    next_pos, finished = current_pos.move(maze)
    if finished:
        break
    visited.append(next_pos)
    current_pos = next_pos

print(count_distinct_positions(visited))


4454


## Part 2

In [20]:
maze = process_input(data)

start_pos = get_start_pos(maze)

n_possible_obstructions = 0

for y in tqdm(range(len(maze))):
    for x in range(len(maze[y])):
        if maze[y][x] == '.':
            # replace with # 
            temp_maze = maze.copy()
            temp_maze[y] = temp_maze[y][:x] + '#' + temp_maze[y][x+1:]
            visited = [start_pos]

            current_pos = start_pos
            while True:
                next_pos, finished = current_pos.move(temp_maze)
                if finished:
                    break
                if next_pos in visited:
                    n_possible_obstructions += 1
                    break

                visited.append(next_pos)
                current_pos = next_pos


print(n_possible_obstructions)

  1%|          | 1/130 [04:21<9:22:39, 261.70s/it]


KeyboardInterrupt: 